In [87]:
import numpy as np

# Convert HMS to degrees

In [88]:
def hms2dec(hours, minutes, seconds):
    # 1 hour is 15 degrees
    return 15*(hours + minutes/60 + seconds/(60*60))


hms2dec(23, 12, 6)

348.025

# Convert DMS to decimal degrees

In [89]:
def dms2dec(degrees, arcminutes, arcseconds):
    angle = abs(degrees) + arcminutes/60 + arcseconds/(60*60)
    return angle if degrees > 0 else -angle


print(dms2dec(22, 57, 18))
print(dms2dec(-66, 5, 5.1))

22.955
-66.08475


# HMS to DMS

A minute in HMS is equal to 15 arcminutes in DMS and a second is equal to 15
arcseconds.

# Angular Distance

In [90]:
def angular_dist(r1, d1, r2, d2):
    r1, d1, r2, d2 = map(np.radians, [r1, d1, r2, d2])
    a = np.sin(np.abs(d1 - d2)/2)**2
    b = np.cos(d1)*np.cos(d2)*np.sin(np.abs(r1 - r2)/2)**2
    d = 2*np.arcsin(np.sqrt(a + b))
    return np.degrees(d)
    

print(angular_dist(21.07, 0.1, 21.15, 8.2))
print(angular_dist(10.3, -3, 24.3, -29))

8.100392318146506
29.208498180546595


# Loading Data

In [91]:
def import_bss(file='data1/bss.dat'):
    result = []
    data = np.loadtxt(file, usecols=range(1, 7))
    for i, row in enumerate(data, 1):
        ascension = hms2dec(row[0], row[1], row[2])
        declination = dms2dec(row[3], row[4], row[5])
        result.append((i, ascension, declination))
    return result


def import_super(file='data1/super.csv'):
    result = []
    data = np.loadtxt(file, delimiter=',', skiprows=1,
                      usecols=[0, 1])
    for i, row in enumerate(data, 1):
        ascension = row[0]
        declination = row[1]
        result.append((i, ascension, declination))
    return result


bss_cat = import_bss()
super_cat = import_super()

print(bss_cat)
print(super_cat)

[(1, 1.1485416666666666, -47.60530555555556), (2, 2.6496666666666666, -30.463416666666667), (3, 2.7552916666666665, -26.209194444444442)]
[(1, 1.0583407, -52.9162402), (2, 2.6084425, -41.5005753), (3, 2.7302499, -27.706955)]


# Find closest Point

In [92]:
def find_closest(catalogue, ascension, declination):
    closest = (None, np.inf)
    for item in catalogue:
        distance = angular_dist(ascension, declination, item[1], item[2])
        if distance < closest[1]:
            closest = (item[0], distance)
    return closest


cat = import_bss('data2/bss.dat')
print(find_closest(cat, 175.3, -32.5))

cat = import_bss('data2/bss.dat')
print(find_closest(cat, 32.2, 40.7))

(156, 3.7670580226469053)
(26, 57.729135775621295)


# Cross-matching

In [96]:
def crossmatch(catalogue1, catalogue2, max_dist):
    matches = []
    no_matches = []
    for item1 in catalogue1:
        closest = find_closest(catalogue2, item1[1], item1[2])
        if closest[1] > max_dist:
            no_matches.append(item1[0])
        else:
            matches.append((item1[0], closest[0], closest[1]))
    return matches, no_matches


bss_cat = import_bss('data3/bss.dat')
super_cat = import_super('data3/super.csv')
max_distance = 40/3600  # 40 arcsecond
match, no_match = crossmatch(bss_cat, super_cat, max_distance)
print(match[:3])
print(no_match[:3])
print(len(no_match))

max_distance = 5/3600  # 5 arcsecond
match, no_match = crossmatch(bss_cat, super_cat, max_distance)
print(match[:3])
print(no_match[:3])
print(len(no_match))

[(1, 2, 0.00010988610938710059), (2, 4, 0.0007649845967242494), (3, 5, 0.00020863352870707666)]
[5, 6, 11]
9
[(1, 2, 0.00010988610938710059), (2, 4, 0.0007649845967242494), (3, 5, 0.00020863352870707666)]
[5, 6, 11]
40
